In [1]:
#import necessary tools
import re
from string import digits
import pandas as pd
import numpy as np

In [19]:
#read the documents
with open('data/135LB.txt','rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16").rstrip("\n")
    contents = contents.split("\r\n") #maybe need to strip \t?

In [20]:
img =[]
Choice_RESP = []
Choice_RT=[]


#loop through each line
for line in range(len(contents)-15):
    
    #this line of "running: listlike/black" is in the fixed position of each of 4 conditions (b1b2l1l2)
    #therefore, we use it as our anchor to locate meaningful blocks

    like_check = contents[line+5].startswith("\tRunning: ListReRate") == True
    
    if contents[line] == '\t*** LogFrame Start ***' and like_check:
        
        #loop inside the meaningful block                
        for inner_line in range(15):
            
            new_line = line + inner_line
            
            #mark image
            if contents[new_line].startswith("\timg: "):
                imga = contents[new_line].split("img: ",1)[1] 
                #print("image is", imga)
                img.append(imga)
                
             
            #mark choice Like response 
            if contents[new_line].startswith("\tSlideR2.RESP: ") and contents[new_line].split("\tSlideR2.RESP: ",1) != ['']:

                Choice_RESPa = contents[new_line].split("\tSlideR2.RESP: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Like response time
            if contents[new_line].startswith("\tSlideR2.RT: ") and contents[new_line].split("\tSlideR2.RT: ",1) != ['']:
                
                Choice_RTa = contents[new_line].split("\tSlideR2.RT: ",1)[1]
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa


#Assign variable names  
        
img = [int(i) for i in img] 
img_df = pd.DataFrame(img)
img_df.columns = ['imgL2']

Choice_RESP = [int(i) for i in Choice_RESP] 
Choice_RESP_df = pd.DataFrame(Choice_RESP)
Choice_RESP_df.columns = ['rateL2']

Choice_RT = [int(i) for i in Choice_RT] 
Choice_RT_df = pd.DataFrame(Choice_RT)
Choice_RT_df.columns = ['rtL2']


In [21]:
#merge the data
sub_id = contents[16].split("Subject: ",1)[1] 
sub_df = pd.DataFrame(np.repeat(sub_id, img_df.shape[0], axis = 0))
sub_df.columns = ['sub']


all = pd.concat([sub_df.reset_index(drop=True), img_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RESP_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RT_df], axis=1)

all

,sub,imgL2,rateL2,rtL2
0,1766,3,3,6747
1,1766,4,7,2760
2,1766,5,6,2433
3,1766,6,4,2455
4,1766,7,1,2238
...,...,...,...,...
89,1766,114,3,1534
90,1766,115,1,3370
91,1766,116,2,2071
92,1766,117,5,2013


In [22]:
#remove last NA line if needed

#all_all= all[:-1]

all_all= all
all_all["sub"] = pd.to_numeric(all_all["sub"])



all_all

,sub,imgL2,rateL2,rtL2
0,1766,3,3,6747
1,1766,4,7,2760
2,1766,5,6,2433
3,1766,6,4,2455
4,1766,7,1,2238
...,...,...,...,...
89,1766,114,3,1534
90,1766,115,1,3370
91,1766,116,2,2071
92,1766,117,5,2013


In [23]:

#export
all_all.to_excel("rate2_135.xlsx")

#has wrong first column, needs to manually remove